# Data Inference Demo Tutorial using OpenAI APIs and LangChain

## Setup

In [53]:
import os
from dotenv import load_dotenv

load_dotenv('./.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [54]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

Dataset used: https://www.kaggle.com/datasets/gagandeep16/car-sales

## Using Prompts

In [76]:
import pandas as pd

df = pd.read_csv('./data/car_sales.csv').copy()
print(df.shape)
df.head()

(157, 16)


,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
0,Acura,Integra,16.919,16.360,Passenger,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2/2/2012,58.280150
1,Acura,TL,39.384,19.875,Passenger,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,6/3/2011,91.370778
2,Acura,CL,14.114,18.225,Passenger,NaN,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,1/4/2012,NaN
3,Acura,RL,8.588,29.725,Passenger,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,3/10/2011,91.389779
4,Audi,A4,20.397,22.255,Passenger,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,10/8/2011,62.777639


In [93]:
data_sample = df.head(10).to_csv()

In [55]:
from langchain.llms import OpenAI

llm = OpenAI(model='text-curie-001', temperature=0)

In [304]:
# using chat model

from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

system_template = """/
The following is a conversation between a Human and an AI assistant expert on data visualization with perfect Python 3 syntax. The human will provide a sample dataset for the AI to use as the source. The real dataset that the human will use with the response of the AI is going to have several more rows. The AI assistant will only reply in the following JSON format: 

{{ 
"charts": [{{'title': string, 'chartType': string, 'parameters': {{...}}}}, ... ]
}}

Instructions:

1. chartType must only contain methods of plotly.express from the Python library Plotly.
2. The format for charType string: plotly.express.chartType.
3. For each chartType, parameters must contain the value to be used for all parameters of that plotly.express method.
4. There should 4 parameters for each chart.
5. Do not include "data_frame" in the parameters.
6. There should be {num_charts} charts in total.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = """
Human: 
This is the dataset:

{data}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
prompt = chat_prompt.format_prompt(num_charts=str(10), data=data_sample).to_messages()

In [305]:
print(prompt)

[SystemMessage(content='/\nThe following is a conversation between a Human and an AI assistant expert on data visualization with perfect Python 3 syntax. The human will provide a sample dataset for the AI to use as the source. The real dataset that the human will use with the response of the AI is going to have several more rows. The AI assistant will only reply in the following JSON format: \n\n{ \n"charts": [{\'title\': string, \'chartType\': string, \'parameters\': {...}}, ... ]\n}\n\nInstructions:\n\n1. chartType must only contain methods of plotly.express from the Python library Plotly.\n2. The format for charType string: plotly.express.chartType.\n3. For each chartType, parameters must contain the value to be used for all parameters of that plotly.express method.\n4. There should 4 parameters for each chart.\n5. Do not include "data_frame" in the parameters.\n6. There should be 10 charts in total.\n', additional_kwargs={}), HumanMessage(content='\nHuman: \nThis is the dataset:\n\

In [306]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = chat_llm(prompt)
total_token = cb.total_tokens
total_cost = cb.total_cost

In [307]:
result.content

'{ \n"charts": [\n    {\'title\': \'Scatter Chart\', \'chartType\': \'plotly.express.scatter\', \'parameters\': {\'x\': \'Horsepower\', \'y\': \'Price_in_thousands\', \'color\': \'Manufacturer\', \'size\': \'Sales_in_thousands\'}},\n    {\'title\': \'Bar Chart\', \'chartType\': \'plotly.express.bar\', \'parameters\': {\'x\': \'Manufacturer\', \'y\': \'Sales_in_thousands\', \'color\': \'Vehicle_type\', \'barmode\': \'group\'}},\n    {\'title\': \'Line Chart\', \'chartType\': \'plotly.express.line\', \'parameters\': {\'x\': \'Latest_Launch\', \'y\': \'Sales_in_thousands\', \'color\': \'Manufacturer\', \'line_group\': \'Vehicle_type\'}},\n    {\'title\': \'Histogram\', \'chartType\': \'plotly.express.histogram\', \'parameters\': {\'x\': \'Price_in_thousands\', \'color\': \'Vehicle_type\', \'nbins\': 10}},\n    {\'title\': \'Box Plot\', \'chartType\': \'plotly.express.box\', \'parameters\': {\'x\': \'Manufacturer\', \'y\': \'Price_in_thousands\', \'color\': \'Vehicle_type\', \'notched\': T

In [308]:
exec(f"out = {result.content}")

In [309]:
out

{'charts': [{'title': 'Scatter Chart',
   'chartType': 'plotly.express.scatter',
   'parameters': {'x': 'Horsepower',
    'y': 'Price_in_thousands',
    'color': 'Manufacturer',
    'size': 'Sales_in_thousands'}},
  {'title': 'Bar Chart',
   'chartType': 'plotly.express.bar',
   'parameters': {'x': 'Manufacturer',
    'y': 'Sales_in_thousands',
    'color': 'Vehicle_type',
    'barmode': 'group'}},
  {'title': 'Line Chart',
   'chartType': 'plotly.express.line',
   'parameters': {'x': 'Latest_Launch',
    'y': 'Sales_in_thousands',
    'color': 'Manufacturer',
    'line_group': 'Vehicle_type'}},
  {'title': 'Histogram',
   'chartType': 'plotly.express.histogram',
   'parameters': {'x': 'Price_in_thousands',
    'color': 'Vehicle_type',
    'nbins': 10}},
  {'title': 'Box Plot',
   'chartType': 'plotly.express.box',
   'parameters': {'x': 'Manufacturer',
    'y': 'Price_in_thousands',
    'color': 'Vehicle_type',
    'notched': True}},
  {'title': 'Area Chart',
   'chartType': 'plotly.e

In [310]:
out['charts'][0]

{'title': 'Scatter Chart',
 'chartType': 'plotly.express.scatter',
 'parameters': {'x': 'Horsepower',
  'y': 'Price_in_thousands',
  'color': 'Manufacturer',
  'size': 'Sales_in_thousands'}}

In [325]:
output = {'charts': [{'title': 'Sales by Manufacturer', 'chartType': 'plotly.express.bar', 'parameters': {'x': 'Manufacturer', 'y': 'Sales_in_thousands'}}, {'title': 'Resale Value by Manufacturer', 'chartType': 'plotly.express.box', 'parameters': {'x': 'Manufacturer', 'y': '__year_resale_value'}}, {'title': 'Engine Size vs Horsepower', 'chartType': 'plotly.express.scatter', 'parameters': {'x': 'Engine_size', 'y': 'Horsepower'}}, {'title': 'Fuel Efficiency Distribution', 'chartType': 'plotly.express.histogram', 'parameters': {'x': 'Fuel_efficiency'}}]}

In [330]:
import plotly.express

if True:
    code = f"""
params = {output['charts'][0]['parameters']}
params['data_frame'] = df
params['title'] = "{output['charts'][0]['title']}"

fig = {output['charts'][0]['chartType']}(**params)

fig.show()
    """

exec(code)

In [312]:
code = f"""

params = {out['charts'][1]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][1]['title']}"

fig = {out['charts'][1]['chartType']}(**params)

fig.show()
"""

exec(code)

In [313]:
code = f"""

params = {out['charts'][2]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][2]['title']}"

fig = {out['charts'][2]['chartType']}(**params)

fig.show()
"""

exec(code)

In [314]:
code = f"""

params = {out['charts'][3]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][3]['title']}"

fig = {out['charts'][3]['chartType']}(**params)

fig.show()
"""

exec(code)

In [315]:
code = f"""

params = {out['charts'][4]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][4]['title']}"

fig = {out['charts'][4]['chartType']}(**params)

fig.show()
"""

exec(code)

In [316]:
code = f"""

params = {out['charts'][5]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][5]['title']}"

fig = {out['charts'][5]['chartType']}(**params)

fig.show()
"""

exec(code)

In [317]:
code = f"""

params = {out['charts'][6]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][6]['title']}"

fig = {out['charts'][6]['chartType']}(**params)

fig.show()
"""

exec(code)

In [318]:
code = f"""

params = {out['charts'][7]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][7]['title']}"

fig = {out['charts'][7]['chartType']}(**params)

fig.show()
"""

exec(code)

In [319]:
code = f"""

params = {out['charts'][8]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][8]['title']}"

fig = {out['charts'][8]['chartType']}(**params)

fig.show()
"""

exec(code)

c:\ProgramData\Anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\ProgramData\Anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [320]:
code = f"""

params = {out['charts'][9]['parameters']}
params['data_frame'] = df
params['title'] = "{out['charts'][9]['title']}"

fig = {out['charts'][9]['chartType']}(**params)

fig.show()
"""

exec(code)